In [1]:
import sympy as sp # imports symbolic math library
from sympy.plotting import plot_parametric
#%pylab inline 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import animation
#plt.switch_backend('tkagg')
#print plt.get_backend()

x, z, t = sp.symbols('x z t')

In [2]:
def init():
    for line in lines:
        line.set_data([], [])
    particles.set_data([], [])
    return lines[0], lines[1], particles

In [3]:
def wavet(a, k, w):
    eta = sp.lambdify((x,t), a*sp.cos(k*x-w*t), 'numpy')
    return eta

In [4]:
def velocity_fluid(a,k,w,H):
    u_x = sp.lambdify((x,z,t), a*w*sp.cosh(k*(z+H))/sp.sinh(k*H)*sp.cos(k*x-w*t), 'numpy')
    u_z = sp.lambdify((x,z,t), a*w*sp.sinh(k*(z+H))/sp.sinh(k*H)*sp.sin(k*x-w*t), 'numpy')
    return u_x, u_z; 

In [5]:
def particle_orbit(t_start, t_end, x_start,z_start):
    time     = np.arange(t_start, t_end, dt)
    N        = len(time)
    x_particle = np.zeros(N)
    z_particle = np.zeros(N)
    x_particle[0] = x_start
    z_particle[0] = z_start
    for i in range(N-1):
        x_particle[i+1] = x_particle[i]+u_x(x_particle[i],z_particle[i],time[i])*(time[i+1]-time[i]) # x_particle[0],z_particle[0] to get closed orbit
        z_particle[i+1] = z_particle[i]+u_z(x_particle[i],z_particle[i],time[i])*(time[i+1]-time[i]) # x_particle[0],z_particle[0] to get closed orbit
    return x_particle, z_particle

In [6]:
def stokes_drift(t_start, Period, x_start,z_start):
    x, y = particle_orbit(t_start,t_start+Period,x_start,z_start)
    s_d  = (x[len(x)-1]-x_start)/Period
    s_d_t= a**2*w*k*np.cosh(2*k*(z_start+H))/(2*np.sinh(k*H)**2)
    print s_d, s_d_t
    return s_d, s_d_t

In [7]:
a        = 0.2
k        = 2.0
w        = 2.0
H        = 10.0
Lambda   = 2*np.pi/k
t_start  = 0.0
Period   = 2*np.pi/w
x_start  = 0.0
z_start  = a
dt       = 0.01


In [8]:
u_x, u_z=velocity_fluid(a,k,w,H)
eta     =wavet(a,k,w)

t_end = t_start + 10*dt
t_end_m    = t_start+2*np.pi/w

s_d, s_dt = stokes_drift(t_start,Period,x_start,z_start)
#x, y = particle_orbit(t_start,t_start+4.0,x_start,z_start)


0.110593803236 0.356086548559


In [9]:
fig = plt.figure()
ax = plt.axes(aspect='equal', autoscale_on=False,xlim=(-Lambda, Lambda), ylim=(-H/2., 10*a))
N = 2
lines = [plt.plot([], [])[0] for _ in range(N)]
particles, = ax.plot([], [], 'bo', ms=6)

def animate(i):
    x, y = particle_orbit(t_start,t_start+(10*i+1)*dt,x_start,z_start)
    xw   = np.arange(-Lambda,Lambda,0.02)
    yw   = eta(xw,t_start+(10*i+1)*dt)

    for j,line in enumerate(lines):
      if j==0:
           line.set_data(x,y)
      if j==1:
           line.set_data(xw,yw)

    particles.set_data(x[len(x)-1], y[len(x)-1])

    return lines[0], lines[1], particles

anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=200, interval=2, blit=True)
#plt.show()
#anim.save('basic_animation.mp4', fps=30)
#display_animation(anim)

In [10]:
from tempfile import NamedTemporaryFile

VIDEO_TAG = """<video controls>
 <source src="data:video/x-m4v;base64,{0}" type="video/mp4">
 Your browser does not support the video tag.
</video>"""

def anim_to_html(anim):
    if not hasattr(anim, '_encoded_video'):
        with NamedTemporaryFile(suffix='.mp4') as f:
            anim.save(f.name, fps=20, extra_args=['-vcodec', 'libx264'])
            video = open(f.name, "rb").read()
        anim._encoded_video = video.encode("base64")
    
    return VIDEO_TAG.format(anim._encoded_video)

from IPython.display import HTML

def display_animation(anim):
    plt.close(anim._fig)
    return HTML(anim_to_html(anim))

display_animation(anim)